# Automated ML

Import Dependencies. In the cell below, import all the dependencies that I will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
import os
import joblib
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
import requests
import json

## Dataset

### Overview

I use the Heart-Failure Dataset from Kaggle and the task is to predict the mortality of the patients based on their clinical records. I create the workspace and experiment in the Azure ML studio. I check if the computer target already exists. If it does not then it creates one. Finally, I display the first rows of the Heart-Failure Dataset.

In [ ]:
#Create a workspace and an experiment in Azureml
ws = Workspace.from_config()
experiment_name = 'Heart-Failure-AutoMlProject'
project_folder = './Heart-Failure-project'

experiment=Experiment(ws, experiment_name)

#Check if the cluster exists if it does not then create one.
# choose a name for your cluster
cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

In [ ]:
# Get Heart Failure Dataset and add it in pandas
ds = Dataset.get_by_name(ws, name='Heart-Failure')
data = ds.to_pandas_dataframe()

In [ ]:
#Review the first 5 rows in the dataset
data.take(5)

## AutoML Configuration
It is used an AutoML for the classification problem using the Heart Failure Dataset. The AutoML run will look for the best model in relation to the accuracy. The target feature is the "DEATH_EVENT". The experiment timeout is 30 mins and the maximum number of iterations which are executed in parallel are 5. 

In [ ]:
# AutoML settings
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# AutoML config
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=ds,
                             label_column_name="DEATH_EVENT",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
# Submit experiment
Automl_run = experiment.submit(automl_config)

## Run Details

`RunDetails` widget is used to show the different experiments.

In [ ]:
RunDetails(Automl_run).show()

## Best Model

Getb the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = Automl_run.get_output()
print(best_run)
print(fitted_model)

In [ ]:
#TODO: Save the best model
joblib.dump(fitted_model, 'outputs/automlmodel.pkl')
automl_model = Model.register(model_path = 'outputs/automlmodel1.pkl', model_name = "auto-ml", workspace = ws)

## Model Deployment


Remember you have to deploy only one of the two models you trained. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
#Create an Environment 
env = Environment('my-environment')
#Set up the inference_config
inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)
#Set up the deployment_config as webservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True)

#Deploy the model
service = Model.deploy(ws, "myservice", [automl_model], inference_config, deployment_config)

#wait until deployment is complete
service.wait_for_deployment(show_output = True)

#Print the state
print(service.state)

#Print the scoring uri of the service
print(service.scoring_uri)

#Print the primary authentication key for the service
primary, secondary = service.get_keys()
print(primary)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# URL for the web service
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
key = primary

# Two sets of data to score, so we get two results back
data = {"data":
        [
            [
            "age": 17
            "anaemia": 1
            "creatinine_phosphokinase": 452
            "diabetes": 0
            "ejection_fraction": 30
            "high_blood_pressure": 1
            "platelets": 263358.03
            "serum_creatinine": 1.1
            "serum_sodium": 137
            "sex": 0
            "smoking": 0
            "time": 8
            ],
        }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()